In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot", "meeseeksmachine", "renovate", "now", "todo"]

In [4]:
# Parameters
end_date = "2019-07-24"
start_date = "2019-06-24"
renderer = "kaggle"
update_data = True


In [5]:
comments = pd.read_csv('./data/comments.csv', index_col=0).query('author not in @bot_names')
issues = pd.read_csv('./data/issues.csv', index_col=0).query('author not in @bot_names')
prs = pd.read_csv('./data/prs.csv', index_col=0).query('author not in @bot_names')

In [6]:
alt.renderers.enable(renderer);

def my_theme():
    return {
        'config': {
            'axisLeft': {
                'labelFontSize': 15,
            },
            'axisBottom': {
                'labelFontSize': 15,
            },
        }
    }
alt.themes.register('my_theme', my_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [7]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [8]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [9]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/nteract](https://github.com/nteract)
* [github.com/ipython](https://github.com/ipython)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/binder-examples](https://github.com/binder-examples)

In [10]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-06-24** to **2019-07-24**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [11]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 30 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [12]:
merged = prs.query('state == "MERGED"')

In [13]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                  org                         repo  author
  132          nteract                      nteract     395
  121          nteract                    bookstore      72
  126          nteract                     hydrogen      63
  134          nteract                    papermill      59
  135          nteract                    scrapbook      21
  131          nteract                          nes      18
  138          nteract                         vdom      11
  129          nteract              meeting-minutes       9
  133          nteract                   nteract.io       9
  136          nteract                     semiotic       9
  123          nteract                         docs       8
  122          nteract                     commuter       6
  124          nteract                     examples       4
  137          nteract                semiotic-docs       4
  139          nteract                vega-embed-v3       2
  125          nteract   graphql-schema-exploration       1
  127          nteract                          ion       1
  128          nteract                 kernel-relay       1
  130          nteract               minimal-plotly       1
  98        jupyterlab                   jupyterlab     584
  103       jupyterlab               jupyterlab-git      62
  97        jupyterlab            jupyter-renderers      21
  113       jupyterlab               jupyterlab-toc      18
  100       jupyterlab        jupyterlab-commenting      15
  101       jupyterlab     jupyterlab-data-explorer      14
  99        jupyterlab          jupyterlab-celltags      13
  111       jupyterlab         jupyterlab-statusbar      13
  115       jupyterlab            jupyterlab_server      13
  107       jupyterlab  jupyterlab-metadata-service      12
  110       jupyterlab        jupyterlab-shortcutui      10
  ..               ...                          ...     ...
  52           jupyter                    terminado       2
  23           jupyter                accessibility       1
  24           jupyter   cookiecutter-docker-stacks       1
  29           jupyter                         help       1
  41           jupyter           nbconvert-examples       1
  49           jupyter     sphinxcontrib_github_alt       1
  50           jupyter                      surveys       1
  19           ipython                      ipython     203
  17           ipython                    ipykernel      21
  22           ipython                    traitlets      10
  18           ipython                  ipyparallel       8
  20           ipython             ipython-in-depth       7
  21           ipython              ipython-website       4
  2    binder-examples             continuous-build       4
  0    binder-examples                  apt_install       3
  3    binder-examples                   data-quilt       2
  8    binder-examples                        latex       2
  10   binder-examples          multi-language-demo       2
  1    binder-examples                        conda       1
  4    binder-examples                   demo-julia       1
  5    binder-examples           getting-data-xroot       1
  6    binder-examples                  jupyter-sos       1
  7    binder-examples               jupyter-stacks       1
  9    binder-examples           minimal-dockerfile       1
  11   binder-examples                          nix       1
  12   binder-examples             python-conda_pip       1
  13   binder-examples                r_with_python       1
  14   binder-examples                 requirements       1
  15   binder-examples                     setup.py       1
  16   binder-examples                   stencila-r       1
  
  [140 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nteract', 'bookstore', 'hydrogen', 'papermill', 'scrapbook', 'nes', 'vdom', 'meeting-minutes

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [14]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [15]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [16]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:            username  authored authorAssociation
    2           betatim         5            MEMBER
    11           kannes         3       CONTRIBUTOR
    12      matheusmota         2              NONE
    7           davidar         2       CONTRIBUTOR
    0            BoPeng         1      COLLABORATOR
    10       jamesmyatt         1       CONTRIBUTOR
    15          sebkopf         1            MEMBER
    14          parente         1       CONTRIBUTOR
    13  matthewfeickert         1       CONTRIBUTOR
    8         emigre459         1       CONTRIBUTOR
    9         fomightez         1       CONTRIBUTOR
    1        benmarwick         1       CONTRIBUTOR
    6      davidanthoff         1       CONTRIBUTOR
    5           cranmer         1       CONTRIBUTOR
    4         choldgraf         1            MEMBER
    3         bsyouness         1       CONTRIBUTOR
    16        yuvipanda         1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:             username  authored authorAssociation
    3            Carreau        82            MEMBER
    79             minrk        16            MEMBER
    9     LucianaMarques         7       CONTRIBUTOR
    80           mpenkov         4       CONTRIBUTOR
    107         tonyfast         4       CONTRIBUTOR
    61            kd2718         4       CONTRIBUTOR
    98         switowski         3       CONTRIBUTOR
    7              Jamim         3       CONTRIBUTOR
    69            luzpaz         3       CONTRIBUTOR
    52      hongshaoyang         3       CONTRIBUTOR
    46      flying-sheep         3       CONTRIBUTOR
    21           anntzer         3       CONTRIBUTOR
    19    alyssawhitwell         3       CONTRIBUTOR
    18              alok         2       CONTRIBUTOR
    25         blink1073         2            MEMBER
    53           hroncok         2       CONTRIBUTOR
    50          gpotter2         2       CONTRIBUTOR
    87          ppmfloss         2       CONTRIBUTOR
    76            mgeier         2       CONTRIBUTOR
    34        dan-coates         2       CONTRIBUTOR
    12        OriolAbril         2       CONTRIBUTOR
    109        westurner         2              NONE
    38            ellert         2       CONTRIBUTOR
    42         fchapoton         2       CONTRIBUTOR
    2            BoboTiG         2       CONTRIBUTOR
    35      danielballan         2       CONTRIBUTOR
    83        oscar6echo         1       CONTRIBUTOR
    82         ndtallant         1       CONTRIBUTOR
    84           parente         1       CONTRIBUTOR
    85          pickfire         1       CONTRIBUTOR
    0      AhmedRedaAmin         1       CONTRIBUTOR
    81              n8vm         1       CONTRIBUTOR
    78           minho42         1       CONTRIBUTOR
    86        pierstitus         1       CONTRIBUTOR
    75          memeplex         1       CONTRIBUTOR
    74         mbbroberg         1       CONTRIBUTOR
    73        matangover         1       CONTRIBUTOR
    72       martinRenou         1       CONTRIBUTOR
    71             mapio         1       CONTRIBUTOR
    70            maapxa         1       CONTRIBUTOR
    77         michael-k         1       CONTRIBUTOR
    91    

In [17]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org                 repo   username  closed
    0   binder-examples          apt_install    betatim       2
    1   binder-examples          apt_install  yuvipanda       1
    2   binder-examples                conda  choldgraf       1
    3   binder-examples     continuous-build    betatim       2
    4   binder-examples     continuous-build      minrk       1
    5   binder-examples     continuous-build  yuvipanda       1
    6   binder-examples           data-quilt    betatim       1
    7   binder-examples           data-quilt  choldgraf       1
    8   binder-examples           demo-julia    betatim       1
    9   binder-examples   getting-data-xroot  choldgraf       1
    10  binder-examples          jupyter-sos  choldgraf       1
    11  binder-examples       jupyter-stacks  yuvipanda       1
    12  binder-examples                latex  choldgraf       2
    13  binder-examples   minimal-dockerfile      minrk       1
    14  binder-examples  multi-language-demo  choldgraf       2
    15  binder-examples                  nix    betatim       1
    16  binder-examples     python-conda_pip  choldgraf       1
    17  binder-examples        r_with_python    betatim       1
    18  binder-examples         requirements    betatim       1
    19  binder-examples             setup.py    betatim       1
    20  binder-examples           stencila-r      nuest       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:         org              repo     username  closed
    21  ipython         ipykernel      Carreau       5
    22  ipython         ipykernel    blink1073       3
    23  ipython         ipykernel        minrk      13
    24  ipython       ipyparallel        minrk       8
    25  ipython           ipython      Carreau     180
    26  ipython           ipython       ivanov       1
    27  ipython           ipython  meeseeksdev       3
    28  ipython           ipython        minrk      16
    29  ipython           ipython       rgbkrk       1
    30  ipython           ipython    takluyver       2
    31  ipython  ipython-in-depth      Carreau       2
    32  ipython  ipython-in-depth    blink1073       3
    33  ipython  ipython-in-depth        minrk       1
    34  ipython  ipython-in-depth       rgbkrk       1
    35  ipython   ipython-website      Carreau       3
    36  ipython   ipython-website    blink1073       1
    37  ipython         traitlets        minrk       5
    38  ipython         traitlets     rmorshea       5,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:          org                        repo       username  closed
    39   jupyter               accessibility      ellisonbg       1
    40   jupyter  cookiecutter-docker-stacks        parente       1
    41   jupyter               docker-stacks          minrk       9
    42   jupyter               docker-stacks        parente      59
    43   jupyter       enhancement-proposals        Zsailer       1
    44   jupyter       enhancement-proposals   captainsafia       1
    45   jupyter       enhancement-proposals        parente       2
    46   jupyter       enhancement-proposals       willingc       3
    47   jupyter          enterprise_gateway      akchinSTC       3
    48   jupyter          enterprise_gateway    kevin-bates      47
    49   jupyter          enterprise_gateway       lresende      46
    50   jupyter                  governance  SylvainCorlay       1
    51   jupyter    

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [18]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [19]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0           ipython                    ipykernel       2   
  1           ipython                  ipyparallel       1   
  2           ipython                      ipython       6   
  3           ipython             ipython-in-depth       1   
  4           jupyter                accessibility       1   
  5           jupyter                docker-stacks       6   
  6           jupyter           enterprise_gateway       3   
  7           jupyter                      jupyter       3   
  8           jupyter                 jupyter-book       9   
  9           jupyter                jupyter-drive       1   
  10          jupyter               jupyter-sphinx       7   
  11          jupyter               jupyter_client       2   
  12          jupyter                 jupyter_core       1   
  13          jupyter               jupyter_server       2   
  14          jupyter                        nb2kg       1   
  15          jupyter                    nbconvert       6   
  16          jupyter                       nbdime       3   
  17          jupyter                     nbgrader       5   
  18          jupyter                     notebook      26   
  19          jupyter                    qtconsole       1   
  20          jupyter                  repo2docker      12   
  21          jupyter                    telemetry       1   
  22          jupyter                     testpath       1   
  23  jupyter-widgets                   ipyleaflet       4   
  24  jupyter-widgets                   ipywidgets      29   
  25  jupyter-widgets           jupyterlab-sidecar       3   
  26       jupyterhub                 batchspawner       3   
  27       jupyterhub                    binderhub      10   
  28       jupyterhub                dockerspawner       4   
  29       jupyterhub                   helm-chart       1   
  ..              ...                          ...     ...   
  32       jupyterhub                   jupyterhub      17   
  33       jupyterhub        kerberosauthenticator       1   
  34       jupyterhub                  kubespawner       3   
  35       jupyterhub            ldapauthenticator       5   
  36       jupyterhub          mybinder.org-deploy       4   
  37       jupyterhub          nativeauthenticator       1   
  38       jupyterhub               oauthenticator       2   
  39       jupyterhub               systemdspawner       3   
  40       jupyterhub                 team-compass       6   
  41       jupyterhub      the-littlest-jupyterhub      11   
  42       jupyterhub                traefik-proxy       2   
  43       jupyterhub                  wrapspawner       1   
  44       jupyterhub       zero-to-jupyterhub-k8s      12   
  45       jupyterlab            jupyter-renderers       2   
  46       jupyterlab                   jupyterlab      69   
  47       jupyterlab          jupyterlab-celltags       3   
  48       jupyterlab        jupyterlab-commenting      12   
  49       jupyterlab     jupyterlab-data-explorer      11   
  50       jupyterlab               jupyterlab-git       6   
  51       jupyterlab  jupyterlab-metadata-service       1   
  52       jupyterlab               jupyterlab-toc       1   
  53       jupyterlab              jupyterlab_apod       1   
  54       jupyterlab               jupyterlab_app       1   
  55       jupyterlab                 team-compass       4   
  56          nteract                    bookstore       3   
  57          nteract                  coffee_boat       1   
  58          nteract                     hydrogen       7   
  59          nteract                      nteract       4   
  60          nteract                    papermill       7   
  61          nteract                     semiotic       4   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                      ipython/ipyparallel 

In [20]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                 demo-julia       1   
  1           ipython                  ipykernel       1   
  2           ipython                    ipython      19   
  3           ipython                  traitlets       1   
  4           jupyter              docker-stacks       2   
  5           jupyter         enterprise_gateway       6   
  6           jupyter               jupyter-book       6   
  7           jupyter             jupyter-sphinx      10   
  8           jupyter          jupyter.github.io       1   
  9           jupyter             jupyter_client       6   
  10          jupyter             jupyter_server       1   
  11          jupyter                      nb2kg       2   
  12          jupyter                  nbconvert       2   
  13          jupyter                     nbdime       1   
  14          jupyter                   nbgrader       1   
  15          jupyter                   nbviewer       3   
  16          jupyter                   notebook      14   
  17          jupyter                  qtconsole       3   
  18          jupyter                repo2docker       6   
  19          jupyter                  telemetry       3   
  20  jupyter-widgets                 ipyleaflet      10   
  21  jupyter-widgets                 ipywidgets      19   
  22  jupyter-widgets         jupyterlab-sidecar       1   
  23  jupyter-widgets               midicontrols       1   
  24  jupyter-widgets                  pythreejs       2   
  25  jupyter-widgets                   tutorial       1   
  26       jupyterhub               batchspawner       1   
  27       jupyterhub                  binderhub       5   
  28       jupyterhub              dockerspawner       3   
  29       jupyterhub       jupyter-server-proxy       1   
  30       jupyterhub                 jupyterhub       5   
  31       jupyterhub   jupyterhub-deploy-docker       1   
  32       jupyterhub      kerberosauthenticator       1   
  33       jupyterhub                kubespawner       3   
  34       jupyterhub        mybinder.org-deploy       4   
  35       jupyterhub             oauthenticator       1   
  36       jupyterhub               team-compass      11   
  37       jupyterhub    the-littlest-jupyterhub       5   
  38       jupyterhub              traefik-proxy       4   
  39       jupyterhub     zero-to-jupyterhub-k8s       8   
  40       jupyterlab  extension-cookiecutter-js       1   
  41       jupyterlab          jupyter-renderers       3   
  42       jupyterlab                 jupyterlab      56   
  43       jupyterlab   jupyterlab-data-explorer       3   
  44       jupyterlab             jupyterlab-git       4   
  45       jupyterlab             jupyterlab-toc       2   
  46          nteract                  bookstore       5   
  47          nteract                   commuter       1   
  48          nteract                   hydrogen      31   
  49          nteract                    nteract      14   
  50          nteract                  papermill       6   
  51          nteract                  scrapbook       1   
  52          nteract                   semiotic       4   
  53          nteract                       vdom       1   
  
                                  org/repo  
  0             binder-examples/demo-julia  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                   jupyter/jupyter-book  
  7                 jupyter/jupyter-sphinx  
  8              jupyter/jupyter.github.io  
  9                 jupyter/jupyter_client  
  10                jupyter/jupyter_server  
  11                         jupyter/nb2kg  
  12                     jupyter/nbconvert  
  13                        jupyter/nbdime  
  14    

In [21]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [22]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                         repo     kind  count
    45   jupyterlab            jupyter-renderers  created    2.0
    46   jupyterlab                   jupyterlab  created   69.0
    47   jupyterlab          jupyterlab-celltags  created    3.0
    48   jupyterlab        jupyterlab-commenting  created   12.0
    49   jupyterlab     jupyterlab-data-explorer  created   11.0
    50   jupyterlab               jupyterlab-git  created    6.0
    51   jupyterlab  jupyterlab-metadata-service  created    1.0
    52   jupyterlab               jupyterlab-toc  created    1.0
    55   jupyterlab                 team-compass  created    4.0
    70   jupyterlab    extension-cookiecutter-js  created    0.0
    119  jupyterlab            jupyter-renderers   closed    3.0
    120  jupyterlab                   jupyterlab   closed   56.0
    121  jupyterlab          jupyterlab-celltags   closed    0.0
    122  jupyterlab        jupyterlab-commenting   closed    0.0
    123  jupyterlab     jupyterlab-data-explorer   closed    3.0
    124  jupyterlab               jupyterlab-git   closed    4.0
    125  jupyterlab  jupyterlab-metadata-service   closed    0.0
    126  jupyterlab               jupyterlab-toc   closed    2.0
    129  jupyterlab                 team-compass   closed    0.0
    144  jupyterlab    extension-cookiecutter-js   closed    1.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:             org                     repo     kind  count
    27   jupyterhub                binderhub  created   10.0
    28   jupyterhub            dockerspawner  created    4.0
    32   jupyterhub               jupyterhub  created   17.0
    34   jupyterhub              kubespawner  created    3.0
    35   jupyterhub        ldapauthenticator  created    5.0
    36   jupyterhub      mybinder.org-deploy  created    4.0
    40   jupyterhub             team-compass  created    6.0
    41   jupyterhub  the-littlest-jupyterhub  created   11.0
    42   jupyterhub            traefik-proxy  created    2.0
    44   jupyterhub   zero-to-jupyterhub-k8s  created   12.0
    101  jupyterhub                binderhub   closed    5.0
    102  jupyterhub            dockerspawner   closed    3.0
    106  jupyterhub               jupyterhub   closed    5.0
    108  jupyterhub              kubespawner   closed    3.0
    109  jupyterhub        ldapauthenticator   closed    0.0
    110  jupyterhub      mybinder.org-deploy   closed    4.0
    114  jupyterhub             team-compass   closed   11.0
    115  jupyterhub  the-littlest-jupyterhub   closed    5.0
    116  jupyterhub            traefik-proxy   closed    4.0
    118  jupyterhub   zero-to-jupyterhub-k8s   closed    8.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyterhub',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [23]:
# Add comment count data to issues and PRs
comment_counts = comments.groupby(['org', 'repo', 'issue_id']).count().iloc[:, 0].to_frame()
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [24]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), i_issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        
        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [25]:
commentors = (
    comments
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [26]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                  author  count authorAssociation
    14            choldgraf     54            MEMBER
    11              betatim     36            MEMBER
    41                vsoch     10       CONTRIBUTOR
    31      matthewfeickert     10              NONE
    39             stwunsch      8              NONE
    27               kannes      7       CONTRIBUTOR
    17          cornhundred      5              NONE
    33               mrjoh3      4              NONE
    30          matheusmota      4              NONE
    13             cboettig      4            MEMBER
    23      gedankenstuecke      3            MEMBER
    42            yuvipanda      3            MEMBER
    19         davidanthoff      3       CONTRIBUTOR
    28              marskar      3              NONE
    2                BoPeng      3      COLLABORATOR
    9                akarve      3              NONE
    24           jamesmyatt      2       CONTRIBUTOR
    25              jhamman      2              NONE
    1   3keepmovingforward3      2              NONE
    29  mathematicalmichael      2              NONE
    16             coatless      2              NONE
    37              sebkopf      2            MEMBER
    8         YannickJadoul      2              NONE
    7          RaoOfPhysics      2       CONTRIBUTOR
    40         taylorreiter      2              NONE
    34                nuest      1            MEMBER
    32                minrk      1            MEMBER
    36             schluenz      1              NONE
    38            statOmics      1              NONE
    35          robmarkcole      1              NONE
    0                0lorak      1              NONE
    26              jzf2101      1            MEMBER
    22            fomightez      1              NONE
    20         eddelbuettel      1              NONE
    18              cranmer      1       CONTRIBUTOR
    15            chrisburr      1              NONE
    12            bsyouness      1       CONTRIBUTOR
    10               bentut      1              NONE
    6            RAJohansen      1              NONE
    5                 NikI0      1              NONE
    4       MarijaStanojcic      1              NONE
    3             Krastanov      1              NONE
    21               ewwink      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                author  count authorAssociation
    17            Carreau    744            MEMBER
    463             minrk     86            MEMBER
    58     LucianaMarques     85       CONTRIBUTOR
    537          rmorshea     65      COLLABORATOR
    587         takluyver     36            MEMBER
    574            stas00     25              NONE
    440       martinRenou     23              NONE
    203         blink1073     22            MEMBER
    586         tacaswell     20       CONTRIBUTOR
    449       meeseeksdev     19       CONTRIBUTOR
    216        ccordoba12     19       CONTRIBUTOR
    316          gpotter2     16       CONTRIBUTOR
    333           hroncok     14       CONTRIBUTOR
    63              MSeal     12       CONTRIBUTOR
    610          tonyfast     12       CONTRIBUTOR
    594         techto

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [27]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comments.append(i_comments.loc[ix_min])
first_comments = pd.concat(first_comments, axis=1).T

In [28]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                  author  n_first_responses authorAssociation
    4             choldgraf                 24            MEMBER
    3               betatim                 11            MEMBER
    9                kannes                  3       CONTRIBUTOR
    2          RaoOfPhysics                  2       CONTRIBUTOR
    10      matthewfeickert                  2       CONTRIBUTOR
    11                minrk                  1            MEMBER
    16                vsoch                  1       CONTRIBUTOR
    15         taylorreiter                  1              NONE
    14              sebkopf                  1            MEMBER
    13                nuest                  1            MEMBER
    12               mrjoh3                  1              NONE
    0   3keepmovingforward3                  1              NONE
    1             Krastanov                  1              NONE
    8               jhamman                  1              NONE
    7            jamesmyatt                  1       CONTRIBUTOR
    6       gedankenstuecke                  1            MEMBER
    5               cranmer                  1       CONTRIBUTOR
    17            yuvipanda                  1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                author  n_first_responses authorAssociation
    5             Carreau                288            MEMBER
    206             minrk                 31            MEMBER
    240          rmorshea                 28      COLLABORATOR
    18     LucianaMarques                 16       CONTRIBUTOR
    262         takluyver                 13            MEMBER
    89          blink1073                  6            MEMBER
    98         ccordoba12                  5       CONTRIBUTOR
    256          stonebig                  5       CONTRIBUTOR
    142          gpotter2                  5       CONTRIBUTOR
    261         tacaswell                  4       CONTRIBUTOR
    146           hroncok                  4       CONTRIBUTOR
    199          memeplex                  4       CONTRIBUTOR
    133       flutefreak7                  4              NONE
    211           mpenkov                  4       CONTRIBUTOR
    35         ProximaDas                  3              NONE
    275          tonyfast                  3       CONTRIBUTOR
    229          ppmfloss                  3       CONTRIBUTOR
    254            stas00                  3              NONE
    171            kd2718                  3       CONTRIBUTOR
    153        jamesmyatt                  3              NONE
    155        jasongrout                  3            MEMBER
    118            dlukes                  3              NONE
    65         andportnoy                  2              NONE
    114          dhomeier                  2              NONE
    122           dyj6146                  2              NONE
    200        mezaugusto                  2              NONE
    47      TomAugspurger                  2              NONE
    45      SylvainCorlay                  2            MEMBER
    235          remram44                  2       CONTRIBUTOR
    121        dwhswenson  

In [30]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>